# Object Tracing using SORT and YOLOv5

## Detection

### Extracting Frames from videos

In [1]:
import os
import cv2
import gc

In [2]:
FILE_NAME = 'test1'

IMG_PATH = os.path.join('./Data', 'images', f'{FILE_NAME}')
if not os.path.exists(IMG_PATH):
    os.makedirs(IMG_PATH)

In [3]:
vidcap = cv2.VideoCapture(f'./Data/videos/{FILE_NAME}.mp4')
success, image = vidcap.read()

count = 0
while success:
    if count == 10:
        break
    
    # cv2.imwrite(f"{IMG_PATH}/frame_{int(count)}.png", image) # save frame as PNG file      
    cv2.imwrite(f"{IMG_PATH}/frame_{int(count)}.jpg", image) # save frame as JPG file      
    
    ## Clearing Memory
    del image # does it make a difference though ?
    gc.collect()
    
    success, image = vidcap.read()
    
    # print('Read a new frame: ', success)
    count += 1

In [4]:
## Clearing Memory
del vidcap # doesn't make a lot of difference i think since it's just a object
gc.collect()

0

In [5]:
## Reset Kernel

%reset -f

### Running Detector on the video / image directory

In [6]:
import os
import cv2
import gc

In [7]:
FILE_NAME = 'test1'

IMG_PATH = os.path.join('./Data', 'images', f'{FILE_NAME}')
if not os.path.exists(IMG_PATH):
    os.makedirs(IMG_PATH)

In [8]:
import numpy as np

import torch
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [9]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
    # for filename in os.scandir(folder):
        # if filename.is_dir(): # only available for scandir files
        #     continue
            
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img = cv2.imread(os.path.join(folder, filename))
            
            if img is not None:
                images.append(img[..., ::-1])
                
    return images

In [10]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# PRETRAIN_WEIGHT = 'yolov5m6'

PRETRAIN_WEIGHT = 'custom'
WEIGHT_PATH = './pretrained_weights/yolov5m6.pt'

#### Using Pytorch Hub

In [11]:
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', classes=1, trust_repo=True) # only predict persons class
# model = torch.hub.load('ultralytics/yolov5', PRETRAIN_WEIGHT, device=DEVICE, trust_repo=True)  # load on DEVICE = CUDA/CPU
model = torch.hub.load('ultralytics/yolov5', PRETRAIN_WEIGHT, path=WEIGHT_PATH, device=DEVICE, trust_repo=True)  # load on DEVICE = CUDA/CPU

# model.load_state_dict(torch.load('yolov5s_10cls.pt')['model'].state_dict())
model.to(DEVICE)
print()

Using cache found in /home/adhiraj/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-12-8 Python-3.10.8 torch-1.13.0 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)

Fusing layers... 
YOLOv5m6 summary: 378 layers, 35704908 parameters, 0 gradients
Adding AutoShape... 


In [12]:
## Evaluation Mode
model.eval()

# model.conf = 0.25  # NMS confidence threshold
model.conf = 0.1  # NMS confidence threshold

# model.iou = 0.45  # NMS IoU threshold
model.iou = 0.1  # NMS IoU threshold

model.agnostic = False # NMS class-agnostic (means will detect objects even when no classes ?)
model.multi_label = False  # NMS multiple labels per box

# model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
model.classes = [0]

model.max_det = 1000  # maximum number of detections per image
model.amp = False  # Automatic Mixed Precision (AMP) inference

**Loading individually**

In [13]:
## Load Images from Directory
images = load_images_from_folder(folder=IMG_PATH)

In [14]:
RESULT_PATH = os.path.join('./Results', 'images', f'{FILE_NAME}')
if not os.path.exists(RESULT_PATH):
    os.makedirs(RESULT_PATH)

In [15]:
## Saving Individually

coordinates = []

for i, img in enumerate(images):
    
    with torch.no_grad():
        # results = model(img, size=640) # batch of images
        results = model(img, size=1280) # batch of images
    
    ## Results
    # results.print()
    
    # results.save()
    # results.save(save_dir=f"{RESULT_PATH}/frame_{int(i)}.png")  # or .show()
    results.save(save_dir=f"{RESULT_PATH}")  # or .show()
    
    coordinates.append(results.xyxy[0].detach().cpu().numpy()[..., :4])
    
    del results
    torch.cuda.empty_cache()

# print(results.xyxy[0])  # print img1 predictions (Bounding Box pixels) 
#                   x1           y1           x2           y2   confidence        class
# tensor([[7.50637e+02, 4.37279e+01, 1.15887e+03, 7.08682e+02, 8.18137e-01, 0.00000e+00],
#         [9.33597e+01, 2.07387e+02, 1.04737e+03, 7.10224e+02, 5.78011e-01, 0.00000e+00],
#         [4.24503e+02, 4.29092e+02, 5.16300e+02, 7.16425e+02, 5.68713e-01, 2.70000e+01]])

# print()
# display(results.pandas().xyxy[0])  # img predictions (pandas)

Saved 1 image to Results/images/test12
Saved 1 image to Results/images/test13
Saved 1 image to Results/images/test14
Saved 1 image to Results/images/test15
Saved 1 image to Results/images/test16
Saved 1 image to Results/images/test17
Saved 1 image to Results/images/test18
Saved 1 image to Results/images/test19
Saved 1 image to Results/images/test110
Saved 1 image to Results/images/test111


**Loading in Batch**

In [16]:
## You can also send images in a BATCH

# with torch.no_grad():
#     # results = model(images, size=640) # batch of images
#     results = model(images, size=1280) # batch of images

# # Results
# results.print()
# # results.save()
# results.save(save_dir=f"{RESULT_PATH}/run")  # or .show()
# # results.show()

# display(results.pandas().xyxy[0])  # img predictions (pandas)

# coordinates = np.array(results.xyxy)
# coordinates = [x.detach().cpu().numpy() for x in coordinates]

# del results
# torch.cuda.empty_cache()

# # print(results.xyxy[0])  # print img1 predictions (Bounding Box pixels) 
# #                   x1           y1           x2           y2   confidence        class
# # tensor([[7.50637e+02, 4.37279e+01, 1.15887e+03, 7.08682e+02, 8.18137e-01, 0.00000e+00],
# #         [9.33597e+01, 2.07387e+02, 1.04737e+03, 7.10224e+02, 5.78011e-01, 0.00000e+00],
# #         [4.24503e+02, 4.29092e+02, 5.16300e+02, 7.16425e+02, 5.68713e-01, 2.70000e+01]])

# # print()

**Save coordinates**

In [17]:
RESULT_PATH = os.path.join('./Results', 'coordinates', f'{FILE_NAME}')
if not os.path.exists(RESULT_PATH):
    os.makedirs(RESULT_PATH)

In [18]:
for i in range(len(coordinates)):
    # np.savetxt(f'{RESULT_PATH}/frame_{i}.txt', results.xyxy[i].detach().cpu().numpy()[..., :4], fmt='%.4f')
    
    # np.savetxt(f'{RESULT_PATH}/frame_{i}.txt', coordinates[i], fmt='%d', delimiter=',', newline='\n')
    np.savetxt(f'{RESULT_PATH}/frame_{i}.txt', coordinates[i], fmt='%0.2f', delimiter=',', newline='\n')

#### Cloning YOLOv5 repo